In [127]:
import os
import sys
# print(sys.path)
module_path = os.path.abspath(os.path.join('.'))
# print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)
# print(sys.path)

## Initiate RAY

In [128]:
from ray.rllib.algorithms.a3c import A3CConfig
from ray import tune, air
import ray
from env.environment import MalariaEnvironment

In [3]:
from utils.plot_parameter_utils import *

In [4]:
get_init_theta()

array([0.9, 0.9])

In [5]:
ray.init(ignore_reinit_error=True, num_cpus=10,local_mode=True)
# ray.init()

2023-09-22 06:23:17,934	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8266 


Python version:,3.8.8
Ray version:,2.2.0
Dashboard:,http://127.0.0.1:8266


## Prepare the Environment

In [6]:
from ray.tune.registry import register_env

def env_creator(env_config):
    return MalariaEnvironment()  # return an env instance

register_env("MalariaEnvironment", env_creator)
env = MalariaEnvironment()

config = A3CConfig()

# config = config.training(lr=0.01) #.rollouts(num_rollout_workers=1)
# config = config.framework(framework='tf2')

# Set the config object's env.
config = config.environment(env="MalariaEnvironment")

In [7]:
from scipy.stats import linregress, sem

In [8]:
from ray.tune.logger import pretty_print

config = A3CConfig()
# Print out some default values.
# print(config.sample_async)  
# Update the config object.
config = config.training( 
    lr=tune.grid_search([0.01, 0.001, 0.0001]), 
    entropy_coeff=tune.grid_search([0.01, 0.001, 0.0001])
)
# config = config.training(lr=0.01)
# config = config.training(lr=tune.grid_search([0.001, 0.01, 0.1]))
# config = config.rollouts(num_rollout_workers=1)
config = config.framework()

# Set the config object's env.
config = config.environment(env="MalariaEnvironment") 
# Use to_dict() to get the old-style python config dict
# when running with tune.

#pretty_print(config.to_dict())

# algo = config.build()

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import tempfile
import time

import ray
from ray import train, tune
from ray.tune.schedulers import PopulationBasedTraining
from ray.tune.tune_config import TuneConfig
from ray.tune.tuner import Tuner

In [ ]:
# https://docs.ray.io/en/latest/tune/examples/pbt_ppo_example.html
import random

import ray
from ray import air, tune
from ray.tune.schedulers import PopulationBasedTraining

hyperparam_mutations = {
    "lambda": lambda: random.uniform(0.9, 1.0),
    "clip_param": lambda: random.uniform(0.01, 0.5),
    "lr": [1e-3, 5e-4, 1e-4, 5e-5, 1e-5],
#         "num_sgd_iter": lambda: random.randint(1, 30),
#         "sgd_minibatch_size": lambda: random.randint(128, 16384),
#         "train_batch_size": lambda: random.randint(2000, 160000),
}

pbt = PopulationBasedTraining(
    time_attr="time_total_s",
    perturbation_interval=120,
    resample_probability=0.25,
    # Specifies the mutations of these hyperparams
    hyperparam_mutations=hyperparam_mutations,
    #custom_explore_fn=explore,
)

# Stop when we've either reached 100 training iterations or reward=300
stopping_criteria = {"training_iteration": 100, "episode_reward_mean": -20}

tuner = tune.Tuner(
    "A3C",
    tune_config=tune.TuneConfig(
        metric="episode_reward_mean",
        mode="max",
        scheduler=pbt
    ),
    param_space={
        "env": "MalariaEnvironment",
        "kl_coeff": 1.0,
        "num_workers": 4,
        "num_cpus": 64,  # number of CPUs to use per trial
        "num_gpus": 0,  # number of GPUs to use per trial
        "model": {"free_log_std": True},
        # These params are tuned from a fixed starting value.
        "lambda":  tune.grid_search([0.9, 1]),#0.95,
        "clip_param": 0.2,
        "lr": tune.grid_search([1e-3,1e-4]),#1e-4,
        "train_batch_size":  tune.grid_search([10000, 20000])
        # These params start off randomly drawn from a set.
#         "num_sgd_iter": tune.grid_search([10, 20, 30]),
#         "sgd_minibatch_size":  tune.grid_search([128, 512, 2048]),
#         "train_batch_size":  tune.grid_search([10000, 20000, 40000]),
    },
    #run_config=air.RunConfig(stop=stopping_criteria),
)
results = tuner.fit()

/opt/anaconda3/lib/python3.8/site-packages/ray/tune/tune.py:505: UserWarning: Consider boosting PBT performance by enabling `reuse_actors` as well as implementing `reset_config` for Trainable.
  warnings.warn(
:task_name:bundle_reservation_check_func
:actor_name:A3C


<IPython.core.display.HTML object>
:task_name:bundle_reservation_check_func
:actor_name:A3C
:actor_name:RolloutWorker
ACTION OrderedDict([('IRS', array([0.8734805], dtype=float32)), ('ITNS', array([0.9574791], dtype=float32))])
Setting IRS to 0.873480498790741
Setting ITNS to 0.9574791193008423


*** WRITING TO CAMPAIGNS FILE ***
*** SUCCESSFULLY CHANGED CAMPAIGNS FILE*** 


Action is setTrue

Starting Simulation...



:actor_name:RolloutWorker


/root/ray_results/A3C/A3C_MalariaEnvironment_59ada_00000_0_lambda=0.9000,lr=0.0010,train_batch_size=10000_2023-10-06_05-23-34


In [20]:
import pprint

best_result = results.get_best_result()

TypeError: __init__() takes 2 positional arguments but 4 were given

In [19]:


print("Best performing trial's final set of hyperparameters:\n")
pprint.pprint(
    {k: v for k, v in best_result.config.items() if k in hyperparam_mutations}
)

print("\nBest performing trial's final reported metrics:\n")

metrics_to_print = [
    "episode_reward_mean",
    "episode_reward_max",
    "episode_reward_min",
    "episode_len_mean",
]
pprint.pprint({k: v for k, v in best_result.metrics.items() if k in metrics_to_print})

TypeError: __init__() takes 2 positional arguments but 4 were given

In [14]:
df = results.get_dataframe()

In [100]:
import pickle

fileObj = open('/home/mbwie001/ms/objects/a2c_2', 'wb')
pickle.dump(results,fileObj)
fileObj.close()

AttributeError: Can't pickle local object 'AlgorithmConfig.__init__.<locals>.<lambda>'

In [21]:
df

,episode_reward_max,episode_reward_min,episode_reward_mean,episode_len_mean,episodes_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_agent_steps_sampled,...,config/clip_param,config/env,config/kl_coeff,config/lambda,config/lr,config/num_cpus,config/num_gpus,config/num_workers,config/model/free_log_std,logdir
0,-9.128667,-234.591407,-116.533326,1.0,38,0,4,0,0,3840,...,0.2,MalariaEnvironment,1.0,0.9,0.0010,64,0,4,True,/root/ray_results/A3C/A3C_MalariaEnvironment_a...
1,-9.128667,-234.652593,-84.154123,1.0,38,0,4,0,0,2120,...,0.2,MalariaEnvironment,1.0,1.0,0.0010,64,0,4,True,/root/ray_results/A3C/A3C_MalariaEnvironment_a...
2,-9.128496,-233.367893,-38.861286,1.0,40,0,4,0,0,3480,...,0.2,MalariaEnvironment,1.0,0.9,0.0001,64,0,4,True,/root/ray_results/A3C/A3C_MalariaEnvironment_a...
3,-9.128667,-234.591407,-125.293727,1.0,40,0,4,0,0,2320,...,0.2,MalariaEnvironment,1.0,1.0,0.0001,64,0,4,True,/root/ray_results/A3C/A3C_MalariaEnvironment_a...


In [24]:
for log in (df['logdir']):
    print(log)

/root/ray_results/A3C/A3C_MalariaEnvironment_ae70b_00000_0_lambda=0.9000,lr=0.0010_2023-09-22_06-25-30
/root/ray_results/A3C/A3C_MalariaEnvironment_ae70b_00001_1_lambda=1,lr=0.0010_2023-09-22_13-55-32
/root/ray_results/A3C/A3C_MalariaEnvironment_ae70b_00002_2_lambda=0.9000,lr=0.0001_2023-09-22_18-08-48
/root/ray_results/A3C/A3C_MalariaEnvironment_ae70b_00003_3_lambda=1,lr=0.0001_2023-09-23_00-57-17


In [122]:
len(results)

4

In [126]:
df1 = results[0].metrics_dataframe

TypeError: __init__() takes 2 positional arguments but 4 were given

In [114]:
df1.head()

,episode_reward_max,episode_reward_min,episode_reward_mean,episode_len_mean,episodes_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_agent_steps_sampled,...,sampler_results/sampler_perf/mean_env_wait_ms,sampler_results/sampler_perf/mean_env_render_ms,info/learner/default_policy/learner_stats/cur_lr,info/learner/default_policy/learner_stats/entropy_coeff,info/learner/default_policy/learner_stats/policy_loss,info/learner/default_policy/learner_stats/policy_entropy,info/learner/default_policy/learner_stats/var_gnorm,info/learner/default_policy/learner_stats/vf_loss,info/learner/default_policy/learner_stats/grad_gnorm,info/learner/default_policy/learner_stats/vf_explained_var
0,-9.128667,-234.629453,-125.015471,1.0,45,0,4,0,0,40,...,26285.982530,0.0,0.001,0.01,-3785.1753,28.379200,22.627424,103383.360,40.0,8.940697e-08
1,-9.128667,-234.629453,-133.501650,1.0,43,0,4,0,0,80,...,26744.403441,0.0,0.001,0.01,-4529.1320,28.456583,22.639826,149393.640,40.0,1.043081e-07
2,-9.128667,-234.629453,-117.878714,1.0,39,0,4,0,0,120,...,27120.531692,0.0,0.001,0.01,-2136.3780,28.615942,22.692387,50725.582,40.0,5.960465e-08
3,-9.128667,-234.591407,-111.499195,1.0,39,0,4,0,0,160,...,27469.926756,0.0,0.001,0.01,-3066.7485,28.976936,22.796150,109605.164,40.0,8.940697e-08
4,-9.128667,-234.591407,-118.954389,1.0,42,0,4,0,0,200,...,27705.729094,0.0,0.001,0.01,-3542.7349,29.611834,22.956760,120253.260,40.0,5.960465e-08


In [115]:
%matplotlib inline
fig, ax = plt.subplots()


Figure(432x288)


In [116]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(df1["episode_reward_mean"], label="label", color="red")
plt.savefig('a2c.png')
plt.legend()

Figure(432x288)


In [107]:
def plot_mean_r_history(results, colors, labels, ax=None):
    if ax is None:
        fig, ax = plt.subplots()
    ax.set_title("True function (Mean Reward) value over training iterations")
    ax.set_xlabel("training_iteration")
    ax.set_ylabel("Mean Reward")

    for i in range(len(results)):
        print(i)
        df = results[i].metrics_dataframe
        ax.plot(df["episode_reward_mean"], label=labels[i], color=colors[i])
    ax.legend()

In [117]:
def plot_parameter_history(
    results,
    colors,
    labels,
    perturbation_interval=None,
    fig=None,
    ax=None,
    plot_until_iter=None,
    include_colorbar=True,
):
    if fig is None or ax is None:
        fig, ax = plt.subplots()

    theta_0 = get_init_theta()

    x = np.linspace(-0.2, 1.0, 50)
    y = np.linspace(-0.2, 1.0, 50)
    xx, yy = np.meshgrid(x, y)
    xys = np.transpose(np.stack((xx, yy)).reshape(2, -1))
    contour = ax.contourf(xx, yy, Q_batch(xys).reshape(xx.shape), 20)
    ax.set_xlabel("theta0")
    ax.set_ylabel("theta1")
    ax.set_title("Q(theta)")

    scatters = []
    for i in range(len(results)):
        df = results[i].metrics_dataframe

        # Append the initial theta values to the history
        theta0_history = np.concatenate([[theta_0[0]], df["theta0"].to_numpy()])
        theta1_history = np.concatenate([[theta_0[1]], df["theta1"].to_numpy()])
        training_iters = np.concatenate([[0], df["training_iteration"].to_numpy()])

        if plot_until_iter is None:
            plot_until_iter = len(training_iters)

        scatter = ax.scatter(
            theta0_history[:plot_until_iter],
            theta1_history[:plot_until_iter],
            # Size of scatter point decreases as training iteration increases
            s=100 / ((training_iters[:plot_until_iter] + 1) ** 1 / 3),
            alpha=0.5,
            c=colors[i],
            label=labels[i],
        )
        scatters.append(scatter)
        for i, theta0, theta1 in zip(training_iters, theta0_history, theta1_history):
            if i % (perturbation_interval or 1) == 0 and i < plot_until_iter:
                ax.annotate(i, (theta0, theta1))

        if perturbation_interval is not None:
            theta_history = np.hstack(
                (theta0_history.reshape(-1, 1), theta1_history.reshape(-1, 1))
            )[:plot_until_iter, :]
            arrow_starts, deltas = get_arrows(theta_history, perturbation_interval)
            for arrow_start, delta in zip(arrow_starts, deltas):
                ax.arrow(
                    arrow_start[0],
                    arrow_start[1],
                    delta[0],
                    delta[1],
                    head_width=0.01,
                    length_includes_head=True,
                    alpha=0.25,
                )
    ax.legend(loc="upper left")
    if include_colorbar:
        fig.colorbar(contour, ax=ax, orientation="vertical")
    return scatters

In [118]:
plot_parameter_history(
    results,
    colors,
    labels,
    perturbation_interval=120,
    fig=fig,
    ax=axs[0],
)

TypeError: __init__() takes 2 positional arguments but 4 were given

In [17]:



plot_Q_history(grid_results, colors, labels, ax=axs[1])

TypeError: __init__() takes 2 positional arguments but 4 were given

Figure(936x432)
